# Matrix factorization
*Authors: Pantelis Matsakidis & Asteris Barakopoulos*

*Group 21*

## 1. Data and general goal
The goal of this assignment is to implement and compare the performance of different recommender system methods. More specifically, the interest lies in estimating the rating that a user would give to a specific movie. The data consists of 1,000,209 anonymus ratings of users that joined MovieLens at the year 2000 and three columns that represent the user id, the movie id and the rating which is on a 5-star scale (whole ratings). There is a total of 6,040 users and 3,952 movies with each user having at least 20 ratings. On this notebook, the matrix factorization algorithm with gradient descent is being implemented and compared with the following naive approaches that are already implemented and discussed on a separate notebook: 

- $R_{global}(User,Item)=mean(\text{all ratings})$
- $R_{item}(User,Item)=mean(\text{all ratings for item})$
- $R_{user}(User,Item)=mean(\text{all ratings for user})$
- $R_{user-item}(User,Item)= \alpha*R_{user}(User,Item) + \beta*R_{item}(User,Item) + \gamma$

Where $R(User,Item)$ stands for the estimate of the rating that an active user would give to a certain item. To assess and compare the performance of all approaches, 5-fold cross-validation is being used in order to get a more honest idea about their perfomance on new data. For every approach, the Root mean squared error is being used for assessment and its given by the formula : $$RMSE=\sqrt{\frac{1}{n}\displaystyle\sum_{i=1}^{n} (predicted - true)^2}$$ The naive approaches are characterized that way because of their simplicity. Especially the first three approaches use information from one source only, either the users or the movies (items). So, even though they work up to a certain extend, they have some important limitations. Their results can be seen from the table below : 

 

|                   | $R_{global}$| $R_{user}$| $R_{item}$| $R_{user-item}$|
|-------------------|-------------|-----------|-----------|----------------|
|   Training error  | 1.117       | 1.027     | 0.974     | 0.914          |
|                   |             |           |           |                |
|   Test error      | 1.117       | 1.035     | 0.979     | 0.924          |

## 2. Training using gradient descent

The matrix factorization algorithm takes into account information about both users and movies, similar to the linear model approach ($R_{user-item}$). The main idea is that a matrix $X$ can be approximated as a product of two matrices $U$ and $V$ : $X \approx UM$. Where $U$ is a $6040\times K$ matrix, which means that each row represents a user and every user has a fixed number of features, $K$. So, there are 6040 user vectors stored in matrix $U$. Similarly, matrix $V$ has dimensions $K \times3952$, each column representing a movie vector. The estimated rating that user $i$ would give to movie $j$ is given by : $$\hat{x_{ij}}=\displaystyle\sum_{k=1}^{K} u_{ik}v_{kj}$$ 

Where $k$ is the $k-th$ feature, $u_{ik}$ are the values of $U$ and $v_{kj}$ are the values of $V$. The error is $e_{ij}=x_{ij}-\hat{x_{ij}}$. The goal of this approach is to find the values of $U$ and $V$ such that the sum of squared errors is minimum (equivalent to minimizing the RMSE). This optimization is done using gradient descent, by finding in which direction is the error function decreasing most rapidly and then taking a step towards this direction. Assume that for a training sample $x_{ij}$ we get the approximation $\hat{x_{ij}}$ . The gradient of $e_{ij}^2$ gives us the direction towards maximizing the error function : $$\frac{\partial e_{ij}^2}{\partial u_{ik}}=-2e_{ij}*v_{kj}  ,  \frac{\partial e_{ij}^2}{\partial v_{kj}}=-2e_{ij}*u_{ik} $$ 

So, the values of $U$ and $V$ are updated towards the opposite direction as follows : $$u'_{ik}=u_{ik} + \eta*(2e_{ij}*v_{kj} - \lambda*u_{ik})$$ $$v'_{kj}=v_{kj} + \eta*(2e_{ij}*u_{ik} - \lambda*v_{kj})$$ Where $\eta=0.001$ is the learning rate and $\lambda=0.01$ is the regularization parameter that helps prevent overfitting by not letting the weights get inflated.

## 3. Algorithm summary

- For each fold (1 through 5):
    - Split the dataset to training and test set.
    - Initialize the values of $U$ and $V$ randomly from the standard normal distribution.
    - Loop until the RMSE on the test set has very little difference between two iterations :
        - For every sample $x_{ij}$ in the training data :
            - Calculate the error $e_{ij}$
            - Update the i-th row of $U$ and the j-th column of $V$.
        - Calculate the RMSE on the training and test set.
        - if the absolute difference of the RMSE on the test set, between two iterations is less than 0.0001, go to             the next iteration.

In [ ]:
###Modules
import numpy as np 
import matplotlib.pyplot as plt

#------------------------- Matrix Factorization -----------------------------------#

def MF (K,eta,lambda_) :
    ratings = np.genfromtxt("ratings.dat",names = None,dtype = None,delimiter = "::")
    nfolds = 5
    train_error5 = np.zeros(nfolds)
    test_error5 = np.zeros(nfolds)
    np.random.seed(17)
    seqs=[x%nfolds for x in range(len(ratings))]
    np.random.shuffle(seqs)

    lr = eta #learning rate
    reg = lambda_ #regularization parameter lambda

    for fold in range(nfolds):
        train_sel=np.array([x!=fold for x in seqs])
        test_sel=np.array([x==fold for x in seqs])
        train=ratings[train_sel]
        test=ratings[test_sel]
        ### Initialize U, V and RMSE
        if K == 1:
            U = np.random.randn(6040) #User vector
            V = np.random.randn(3952) #Movie vector
        else:
            U = np.random.randn(6040,K) #User vector
            V = np.random.randn(K,3952) #Movie vector
        RMSE = list()
        RMSE_train = list()
        for iterations in range(10000) :
            if K == 1:
                for i in range(len(train)):
                    user = train[i,0] - 1
                    movie = train[i,1] - 1
                    rating = train[i,2]
                    error = rating - U[user]*V[movie] 
                    U[user] = U[user] + lr*(error*V[movie] - reg*U[user])
                    V[movie] = V[movie] + lr*(error*U[user] - reg*V[movie])
               
                ### COMPUTE RMSE at the test set

        
                test_users = test[:,0] - 1
                test_movies = test[:,1] - 1
                test_ratings = test[:,2]
                predictions = U[test_users]*V[test_movies]
                RMSE.append(np.sqrt(np.mean((test_ratings - predictions)**2)))

                ### COMPUTE RMSE at the training set

                train_users = train[:,0] - 1
                train_movies = train[:,1] - 1
                train_ratings = train[:,2]
                train_predictions = U[train_users]*V[train_movies]
                RMSE_train.append(np.sqrt(np.mean((train_ratings - train_predictions)**2)))
            else :
                for i in range(len(train)):
                    user = train[i,0] - 1
                    movie = train[i,1] - 1
                    rating = train[i,2]
                    error = rating - np.dot(U[user,:],V[:,movie]) 
                    # Update user row from U
                    U[user,:] = U[user,:] + lr*(2*error*V[:,movie] - reg*U[user,:])
                    # Update movie column from V
                    V[:,movie] = V[:,movie] + lr*(2*error*U[user,:] - reg*V[:,movie])
            
                ### Compute RMSE at the test set

                test_users = test[:,0] - 1
                test_movies = test[:,1] - 1
                test_ratings = test[:,2]
                predictions = np.sum(U[test_users,:]*V[:,test_movies].T, axis = 1)
                RMSE.append(np.sqrt(np.mean((test_ratings - predictions)**2)))
        
                ### Compute RMSE at the training set

                train_users = train[:,0] - 1
                train_movies = train[:,1] - 1
                train_ratings = train[:,2]
                train_predictions = np.sum(U[train_users,:]*V[:,train_movies].T, axis = 1)
                RMSE_train.append(np.sqrt(np.mean((train_ratings - train_predictions)**2)))
             
            if len(RMSE) >= 2 :
                conv = abs(RMSE[iterations]-RMSE[iterations-1])
                if conv < 0.0001 :
                    print("Converged at iteration " + str(iterations))
                    break 
        ### Compute Final RMSE on train and test set, for the current fold        
        train_error5[fold] = RMSE_train[-1] #np.sqrt(error_train/len(train))
        test_error5[fold] = RMSE[-1]
        print("Fold " + str(fold) + ": RMSE_train=" + str(train_error5[fold]) + "; RMSE_test=" + str(test_error5[fold]))


    print("Mean error on TRAIN: " + str(np.mean(train_error5)))
    print("Mean error on  TEST: " + str(np.mean(test_error5)))

## 4. Implementation

First, the algorithm is being implemented for $K=1$ feature. That means, each user and each movie is being represented by a single value. This selection of $K$ is causing the algorithm to converge faster, which is reasonable because the number of weights to be optimized is $6040*K + 3952*K = 9992$. The algorithm takes about 67 iterations to reach convergence and the results are presented below :

| Training error | Test error   |
|------|------|
|   0.901  | 0.918|

From the results, it can be concluded that for even one feature, this method already has better results than the linear model that was implemented before. There are no signs of overfitting, since the training error is only slightly lower than the test error, but it would be interesting to also try a different setting for $K$ and observe the performance. 

In [ ]:
MF(1,0.001,0.01)
# K=1
# learning rate = 0.001
# lambda = 0.01

The algorithm is now repeated using $K=5$ features. The number of weights to be optimized now is $6040*5 + 3952*5 = 49960$. From the results table below, one can see that the error has overally decreased in comparison with the previous setting.

| Training error | Test error   |
|------|------|
|   0.826  | 0.884|

In [ ]:
MF(5,0.001,0.01)
# K = 5
# learning rate = 0.001
# lambda = 0.01

Next, we are going to implement the algorithm using $K=20$ features which means way more weights to be optimized compared to the previous settings ($6040*20 + 3952*20 = 199840$). The algorithm now needs about 200 iterations to reach convergence and looking at the results there is a serious problem of overfitting. The test error is even larger than the one we obtained for $K=1$ and the training error is much smaller, meaning that our model has lost its ability to generalize.

| Training error | Test error   |
|------|------|
|   0.75  | 0.98|

In [ ]:
MF(20,0.001,0.01)
# K = 20
# learning rate = 0.001
# lambda = 0.01

Clearly, the $K=20$ setting is not proper for our data and it seems that the best performance is achived for $K=5$. As a conclusion, the matrix factorization method outperforms all of the four naive approaches that were implemented on this assignment. Even though the naive approaches can work at some circumstances and are way faster than matrix factorization, the latter gives more trusted and accurate results especially when all hyperparameters are optimally tuned.